In [12]:
import torch
from torch.nn import functional as F
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, set_seed

c:\Piotr Pijanowski\Studia\Semestr-5\modele_jezykowe\pracownia_2\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
MODEL_NAME = "flax-community/papuGaPT2"
DEVICE = "cuda" if torch.cuda.is_available() else ("mps" if torch.backends.mps.is_available() else "cpu")
SEED = 42

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME).to(DEVICE)

In [14]:
TOPK = 10
TOPP = 0.8



In [15]:
# Otwieranie pliku z prefiskami

with open('prefiksy.txt', 'r', encoding="utf-8") as f:
    prefixes = f.read().split('\n')



In [18]:
def show_next_token_suggestions(prefix, top_k=20):
    # Tokenizacja wejścia
    input_ids = tokenizer(prefix, return_tensors="pt").input_ids.to(DEVICE)

    # Przepuszczenie przez model
    with torch.no_grad():
        outputs = model(input_ids)
    
    # Logity ostatniego tokenu
    logits = outputs.logits[0, -1]

    # Softmax -> prawdopodobieństwa
    probs = torch.softmax(logits, dim=-1)

    # Top-k najlepszych tokenów
    top_probs, top_ids = torch.topk(probs, top_k)

    print(f"\nPrefiks:\n{prefix}\n")
    print(f"Top {top_k} propozycji następnego tokenu:\n")

    for prob, tok_id in zip(top_probs, top_ids):
        token_str = tokenizer.decode([tok_id.item()])
        print(f"{token_str!r}   ->   p = {prob.item():.4f}")

In [19]:
show_next_token_suggestions('Orzeszek jest malutki', top_k=15)


Prefiks:
Orzeszek jest malutki

Top 15 propozycji następnego tokenu:

','   ->   p = 0.5331
' i'   ->   p = 0.1895
'.'   ->   p = 0.0654
' ale'   ->   p = 0.0305
' ('   ->   p = 0.0172
' -'   ->   p = 0.0128
' jak'   ->   p = 0.0122
' a'   ->   p = 0.0087
' ,'   ->   p = 0.0080
' –'   ->   p = 0.0077
' więc'   ->   p = 0.0067
' w'   ->   p = 0.0052
' o'   ->   p = 0.0041
' z'   ->   p = 0.0039
' na'   ->   p = 0.0034
